In [42]:
import numpy as np
def load_sms():
    """
    A wrapper function to load the sms data
    """
    import csv
    lines = []
    hamspam = {'ham': 0, 'spam': 1}
    with open('data/spam.csv', mode='r',encoding='latin-1') as f:
        reader = csv.reader(f)
        # When using the csv reader, each time you use the function
        # next on it, it will spit out a list split at the ','
        header = next(reader)
        # We store this as ("txt",label), where we have used the function
        # hamspam to convert from "ham","spam" to 0 and 1.
        lines = [(line[1],hamspam[line[0]]) for line in reader]

    return lines
spam_no_spam =load_sms()
x1=[]
x2=[]
x3=[]
keywords = ["free", "prize", "win"]
def getx(data,keyword):
    x=[]
    for sms, is_spam in spam_no_spam:
        sms_lower = sms.lower()
        has_keywords = keyword in sms_lower
        if has_keywords:
            x.append('1')
        else:
            x.append('0')
    return x
x1 = getx(spam_no_spam,"free")
x2 = getx(spam_no_spam,"prize")
x3 = getx(spam_no_spam,"win")
problem2_X = np.array([x1,x2,x3],dtype='int').transpose()
def gety(data):
    y=[]
    for sms, is_spam in data:
        if is_spam:
            y.append('1')
        else:
            y.append('0')
    return y
problem2_Y = np.array(gety(spam_no_spam),dtype='int')
def train_test_validation(X,Y,test_size=0.4,validation_size=0.2,random_state=None,shuffle=True):
    """
    Performs a train test validation split of the data [train_data,test_data,validation_data]

    Parameters:
    -----------
    X : The input X, shape (n_samples,n_features)
    Y : The input labells, shape (n_samples)
    test_size : the proportion of data that should be test data
    validation_size : the proportion of data that should be validation data
    random_state : the random state variable passed through to sklearns train_test_split

    Returns:
    ----------
    X_train, X_test, X_valid, Y_train, Y_test, Y_valid

    Examples:
    ----------
    >>> X_train, X_test, X_valid, Y_train, Y_test, Y_valid = train_test_validation(X,Y,test_size=0.25,validation_size=0.25)
    """
    from sklearn.model_selection import train_test_split

    X_train,X_tt,Y_train,Y_tt = train_test_split(X,Y,
                                                 test_size=test_size+validation_size,
                                                 random_state=random_state,
                                                 shuffle=shuffle)
    X_test,X_valid,Y_test,Y_valid = train_test_split(X_tt,Y_tt,
                                                     test_size=(validation_size)/(test_size + validation_size),
                                                     random_state=random_state,
                                                     shuffle=shuffle)

    return X_train, X_test, X_valid, Y_train, Y_test, Y_valid
problem2_X_train,problem2_X_test,problem2_X_calib,problem2_Y_train,problem2_Y_test,problem2_Y_calib = train_test_validation(problem2_X,problem2_Y,test_size = 0.4,validation_size = 0.2)
print(problem2_X_train.shape,problem2_X_calib.shape,problem2_X_test.shape,problem2_Y_train.shape,problem2_Y_calib.shape,problem2_Y_test.shape)


(2228, 3) (1115, 3) (2229, 3) (2228,) (1115,) (2229,)


In [61]:
# Part 2

class ProportionalSpam(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
        
    def logistic(self, z):
        return 1 / (1 + np.exp(-z))
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        #cost = np.transpose(-y)@np.log(h) - np.transpose(1-y)@np.log(1-h) + (l/2)*np.transpose(t[1:])@t[1:]
        #cost = (1/m)*cost
        #return cost
        logist = np.dot(X, coeffs[1:]) + coeffs[0]
        prediction = self.logistic(logist)
        cost = -np.mean(Y * np.log(prediction) + (1 - Y) * np.log(1 - prediction))
        return cost

    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize

        #Use the f above together with an optimization method from scipy
        #to find the coefficients of the model
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1)
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            G = lambda x: np.exp(x)/(1+np.exp(x))
            return np.round(10*G(np.dot(X,self.coeffs[1:])+self.coeffs[0]))/10 # This rounding is to help you with the calibration


In [62]:
try:
    import numpy as np
    test_instance = ProportionalSpam()
    test_loss = test_instance.loss(np.array([[1,0,1],[0,1,1]]),np.array([1,0]),np.array([1.2,0.4,0.3,0.9]))
    assert (np.abs(test_loss-1.2828629432232497) < 1e-6)
    print("Your loss was correct for a test point")
except:
    print("Your loss was not correct on a test point")

Your loss was correct for a test point


In [73]:
# Part 3
from sklearn.tree import DecisionTreeRegressor
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import DecisionTreeClassifier

problem2_ps = ProportionalSpam()
problem2_ps.fit(problem2_X_train, problem2_Y_train)
problem2_X_pred = problem2_model.predict(problem2_X_calib).reshape(-1,1)
problem2_calibrator = CalibratedClassifierCV(base_estimator=DecisionTreeClassifier(), method='sigmoid')
problem2_calibrator.fit(problem2_X_pred, problem2_Y_calib)

/Users/chengu/Desktop/anaconda3/lib/python3.11/site-packages/sklearn/calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


CalibratedClassifierCV(base_estimator=DecisionTreeClassifier())

In [75]:
problem2_final_predictions = problem2_calibrator.predict(problem2_ps.predict(problem2_X_test).reshape(-1, 1))
problem2_01_loss = brier_score_loss(problem2_Y_test, problem2_final_predictions)
alpha = 0.01
n_test = len(problem2_Y_test)
mean_loss = np.mean(problem2_01_loss)
std_error = st.sem(problem2_01_loss)
interval = st.t.interval(1 - alpha, n_test - 1, loc=mean_loss, scale=std_error)

problem2_interval = interval

# Print results
print(f"0-1 Test Loss: {problem2_01_loss}")
print(f"99% Confidence Interval: {problem2_interval}")

NameError: name 'brier_score_loss' is not defined